In [ ]:
import random
import time
import copy
from collections import deque

import numpy as np

import torch as tc
import torch.optim as opt

from model import SnakeNet
from core import CUDA_AVAILABLE, DEVICE
from config import NROW,NCOL,EPISODE_MAXLEN
from per import PER
from env import Env
from util import state2input

import time as timelib
from PIL import Image
from PIL import ImageDraw
from torchvision import transforms
import matplotlib.pyplot as plt
from IPython import display

def render(state,sleep_time=0,clear=True,save_name=None):
    #print(obs)
    txt = Image.new("RGBA", (NCOL*30,NROW*30), (10,50,100,100))
    draw = ImageDraw.Draw(txt)
    grid_size = 30
    for i in range(NROW):
        for j in range(NCOL):
            y1 = i*grid_size
            x1 = j*grid_size
            y2 = (i+1)*grid_size
            x2 = (j+1)*grid_size
            draw.rectangle(((x1, y1), (x2, y2)), outline='black', width=1)
    for y,x in state['foods']:
        draw.ellipse((x*grid_size, y*grid_size, (x+1)*grid_size, (y+1)*grid_size), fill = 'yellow', outline ='yellow')
    for y,x in state['snake']:
        if (y,x)==state['snake'][-1]:
            draw.polygon([((x+1/2)*grid_size,y*grid_size),(x*grid_size,(y+1/2)*grid_size),
                ((x+1/2)*grid_size,(y+1)*grid_size), ((x+1)*grid_size,(y+1/2)*grid_size)], fill = 'red')
        else:
            draw.rectangle(((x*grid_size, y*grid_size), ((x+1)*grid_size, (y+1)*grid_size)), fill='red', outline='red')
    txt = txt.resize((64, 64*NROW//NCOL)).convert('RGB')
    if save_name:
        txt.save(save_name)
    numpy_image = np.array(txt)
    plt.axis("off")
    trans1 = transforms.ToTensor()
    tensor_image = trans1(numpy_image)
    tf = transforms.ToPILImage()
    plt.imshow(tf(tensor_image))
    plt.show()
    if sleep_time:
        timelib.sleep(sleep_time)
    if clear:
        plt.close()
        display.clear_output(wait=False)
    return tensor_image

def stateTransform(state,flipy,flipx,deltay,deltax):
    def fy(y): return (NROW-1-y+deltay if flipy else y+deltay)%NROW
    def fx(x): return (NCOL-1-x+deltax if flipx else x+deltax)%NCOL
    ret=copy.deepcopy(state)
    ret['snake']=[(fy(y),fx(x)) for y,x in state['snake']]
    ret['foods']=[(fy(y),fx(x)) for y,x in state['foods']]
    return ret
def actTransform(act,flipy,flipx):
    if act%2: return (act+2)%4 if flipx else act #1,3
    else: return (act+2)%4 if flipy else act  #0,2
transformOptions=[(i,j,y,x) for i in range(2) for j in range(2) for y in range(NROW) for x in range(NCOL)]


while True:
    net=SnakeNet().cuda() if CUDA_AVAILABLE else SnakeNet()
    net.load_state_dict(tc.load('./netw.pt'))
    net.eval()

    with tc.no_grad():
        env = Env(False)
        state=env.reset()
        # render(state,0.1,False)
        # tf = random.sample(transformOptions,1)[0]
        # print(tf)
        # render(stateTransform(state,*tf),0.1,False)
        rwdsum=0
        while not state['done']:
            fy,fx=np.random.randint(0,2),np.random.randint(0,2)
            res = net(tc.tensor(state2input(stateTransform(state,fy,fx,0,0))).to(DEVICE)).detach().squeeze()
            print(res)
            state,rwd = env.step(actTransform(tc.argmax(res),fy,fx))
            rwdsum+=rwd
            print(state['time'],rwdsum)
            render(state,0.03,True)